In [ ]:
!nvidia-smi

Tue Aug 24 16:46:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
# with zipfile.ZipFile("/dataISEG_np/train_patch32_stride8.zip", 'r') as zip_ref:
#     zip_ref.extractall("data_train")
with zipfile.ZipFile("dataISEG_np/test_patch32_stride8.zip", 'r') as zip_ref:
    zip_ref.extractall("data_test")
# with zipfile.ZipFile("/dataISEG_np/pseudo_data.zip", 'r') as zip_ref:
#     zip_ref.extractall("pseudo_data")

In [ ]:
%cd /content/drive/MyDrive/
%run U_Unet.ipynb

/content/drive/MyDrive/iseg2019


In [ ]:
# !pip install import-ipynb
# import import_ipynb
# !pip install volumentations-3D
# from volumentations import *
# !pip install imjoy scikit-image
#https://github.com/ZFTurbo/volumentations/blob/4e2c73add77d7fdbc5f69d4248c893f6e6aa67dc/volumentations/augmentations/transforms.py#L154
import gc
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
import keras.backend as K
from itertools import product
import os
import scipy.io as sio
from scipy import ndimage
import matplotlib.pyplot as plt

from tqdm import tqdm
from fastprogress import master_bar, progress_bar
import nibabel as nib
import glob 
from mpl_toolkits import mplot3d
from IPython.display import clear_output
from keras.utils import np_utils
from sklearn.feature_extraction.image import extract_patches as sk_extract_patches
from sklearn.model_selection import train_test_split
from skimage import exposure
clear_output()

##Load data

In [ ]:
TRAIN_PATH = "/content/data_train/T1*"
PSEUDO_PATH = "/content/pseudo_data/T1*"
TEST_PATH = "/content/data_test/T1*"
CROP_SIZE = (144,176,160,1)
NUM_CLASS = 4
PATCH_SIZE = (32,32,32,1)
EXTRACTION_STEP = (8,8,8,1)
def normalize(volume):
    """Normalize the volume"""
    min = 0.0
    max = 1000.0
    volume = np.clip(volume, min, max)
    volume = (volume - min) / (max - min)
    return volume

def pre_processing(volume, mask = None):
    volume = normalize(volume)
    volume = CenterCrop(CROP_SIZE).apply(volume)
    volume = volume[:,16:,16:,:]
    if mask is not None:
        mask = CenterCrop(CROP_SIZE).apply(mask)
        mask = mask[:,16:,16:,:]
        return volume, mask
    else :
        return volume
class patchSequence(tf.keras.utils.Sequence):
    def __init__(self, train_path=TRAIN_PATH, test_path= TEST_PATH, 
               batch_size=16,patch_size = PATCH_SIZE,
               shuffle=True, augment=True, typeData = "train"):
        
        try:
            self.listT1name =  glob.glob(train_path) + glob.glob(PSEUDO_PATH) if typeData == "train" else glob.glob(test_path)
        except:
            self.listT1name =  glob.glob(train_path) if typeData == "train" else glob.glob(test_path)
        self.patch_size   = patch_size   
        self.batch_size   = batch_size if typeData == "train" else batch_size*2 # batch size
        self.shuffle      = shuffle             # shuffle bool
        self.augment      = augment if typeData == "train" else False  # augment data bool
        self.typeData = typeData
        self.on_epoch_end()
        print('data length:', len(self.listT1name))
    def __len__(self):
        return int(np.floor(len(self.listT1name) / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.listT1name))
        if self.shuffle and self.typeData == "train":
            np.random.shuffle(self.indexes)   

    def histogram_equalise_func(self,volume):
        squashed_volume = np.reshape(volume, (volume.shape[0], volume.shape[1] * volume.shape[2]))
        img_rescale = exposure.equalize_adapthist(squashed_volume, clip_limit=0.01)
        img_rescale = np.reshape(img_rescale, volume.shape)
        return img_rescale      

    def gen_augmentation(self, volume):
        if tf.cast(np.random.choice([0, 1],p=[0.75, 0.25]), tf.bool):
            volume = exposure.equalize_hist(volume)
        return volume

    def getBatchData(self, list_IDT1):
        volumeT1 = np.zeros((self.batch_size, *self.patch_size))
        volumeT2 = np.zeros((self.batch_size, *self.patch_size))
        mask = np.zeros((self.batch_size, *self.patch_size))
        for i, t1name in enumerate(list_IDT1):
            volumeT1[i]= nib.load(t1name).get_fdata()
            volumeT2[i]= nib.load(t1name.replace("T1","T2")).get_fdata()
            mask[i] = nib.load(t1name.replace("T1","mask")).get_fdata()
        ####################### augmentation data ##############################
            if self.augment == True:
                volumeT1[i] = self.gen_augmentation(volumeT1[i])
                volumeT2[i] = self.gen_augmentation(volumeT2[i])
        volumeT1 = tf.convert_to_tensor(volumeT1, tf.float32)
        volumeT2 = tf.convert_to_tensor(volumeT2, tf.float32)
        mask = tf.convert_to_tensor(mask, tf.float32)
        return {"inputT1": volumeT1,"inputT2": volumeT2}, {"active": mask, "levelset": volumeT1}
        # return volumeT1,volumeT2,mask
    def __getitem__(self, index):
        # selects indices of data for next batch
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        # select data and load images
        list_IDT1 = [self.listT1name[k] for k in indexes]
        return self.getBatchData(list_IDT1)
def plot_2Ddata(x, y, numStart = 0, numEnd = 144):
    for i in range(numStart, numEnd):
        plt.figure(i+1)
        plt.subplot(141),plt.imshow(x[0,:,:,i,0]),plt.title('T1')
        plt.subplot(142),plt.imshow(x[1,:,:,i,0]),plt.title('T2')
        plt.subplot(143),plt.imshow(y[0,:,:,i,0]),plt.title('labelT1')
        plt.subplot(144),plt.imshow(y[1,:,:,i,0]),plt.title('labelT2')
def inverseLabel(label_raw, predicted):
    temp = np.zeros(shape=CROP_SIZE)
    temp[:,16:,16:,:]= predicted
    return PadIfNeeded(label_raw.shape).apply(temp)  
def extract_patches(arr, patch_shape=8, extraction_step=1):
    arr_ndim = arr.ndim
    patch_strides = arr.strides

    slices = [slice(None, None, st) for st in extraction_step]
    indexing_strides = arr[tuple(slices)].strides

    patch_indices_shape = ((np.array(arr.shape) - np.array(patch_shape)) //
                           np.array(extraction_step)) + 1
    
    shape = tuple(list(patch_indices_shape) + list(patch_shape))
    strides = tuple(list(indexing_strides) + list(patch_strides))
    patches = np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)
    npatches = np.prod(patches.shape[:arr_ndim])
    return patches.reshape((npatches, ) + patch_shape)

def reconstruct_volume_avg(patches, expected_shape, extraction_step = (8,8,8,1), num_class = 4):
    v_x, v_y, v_z = expected_shape
    p_x, p_y, p_z = patches.shape[1:4]
    s_x, s_y, s_z = extraction_step[:3]# compute the dimensions of the patches array
    n_x = (v_x - p_x) // s_x + 1
    n_y = (v_y - p_y) // s_y + 1
    n_z = (v_z - p_z) // s_z + 1

    vol = np.zeros((v_x, v_y, v_z, num_class))
    count = np.zeros((v_x, v_y, v_z, num_class))

    for p, (i, j, k) in zip(patches, product(range(n_x), range(n_y), range(n_z))):
        vol[i*s_x:i*s_x+ p_x, j*s_y:j*s_y + p_y, k*s_z:k*s_z + p_z]  +=p
        count[i*s_x:i*s_x+ p_x, j*s_y:j*s_y + p_y, k*s_z:k*s_z + p_z] +=1
    return vol/count

def reconstruct_volume_vote(patches, expected_shape, extraction_step = (8,8,8,1), num_class = 4):
    v_x, v_y, v_z = expected_shape
    p_x, p_y, p_z = patches.shape[1:4]
    s_x, s_y, s_z = extraction_step[:3]# compute the dimensions of the patches array
    n_x = (v_x - p_x) // s_x + 1
    n_y = (v_y - p_y) // s_y + 1
    n_z = (v_z - p_z) // s_z + 1

    vol = np.zeros((v_x, v_y, v_z, num_class))

    for p, (i, j, k) in zip(patches, product(range(n_x), range(n_y), range(n_z))):
        vol[i*s_x:i*s_x+ p_x, j*s_y:j*s_y + p_y, k*s_z:k*s_z + p_z]  += np.where(p<np.max(p, axis = -1,keepdims=True),0, 1)
    return vol     
    

create numpy file from data


In [ ]:
list_T1name =  glob.glob("/dataISEG/iSeg-2017-Training/*T1.hdr")
x_trainT1 = []
x_trainT2 = []
y_train = []
for imageNameT1 in list_T1name:
    T1_weight = nib.load(imageNameT1).get_fdata().astype(np.float32)
    T2_weight = nib.load(imageNameT1.replace("T1", "T2")).get_fdata().astype(np.float32)
    label = nib.load(imageNameT1.replace("T1", "label")).get_fdata().astype(np.float32)
    x_trainT1.append(T1_weight)
    x_trainT2.append(T2_weight)
    y_train.append(label)
x_trainT1 = np.stack(x_trainT1)
x_trainT2 = np.stack(x_trainT2)
y_train = np.stack(y_train)
np.save("dataISEG_np/T1_train.npy",x_trainT1)
np.save("dataISEG_np/T2_train.npy",x_trainT2 )
np.save( "dataISEG_np/y_train.npy",y_train)

In [ ]:
t1 = np.load("dataISEG_np/T1_train.npy")
t2 = np.load("dataISEG_np/T2_train.npy")
mask = np.load("dataISEG_np/y_train.npy")

In [ ]:
def gaussian_noise(img):
    img = img + np.random.normal(0, 0.1, img.shape)
    return np.clip(img, 0, 1)
def get_augmentation(x_trainT1, x_trainT2, y_train):
    t1 = x_trainT1
    t2 = x_trainT2
    label = y_train
    i=1
    for volumeT1, volumeT2, mask in zip(x_trainT1, x_trainT2, y_train):
        volumeT1 = gaussian_noise(volumeT1)
        volumeT2 = gaussian_noise(volumeT2)

        t1 = np.vstack((t1, [volumeT1]))
        t2 = np.vstack((t2, [volumeT2]))
        label = np.vstack((label, [mask]))
    return t1, t2, label
# t1, t2, mask = get_augmentation(x_trainT1, x_trainT2, y_train)
# np.save("dataISEG_np/T1_fullAug.npy", t1)
# np.save("dataISEG_np/T2_fullAug.npy", t2)
# np.save("dataISEG_np/mask_fullAug.npy", mask)
# t1 = np.load("dataISEG_np/T1_fullAug.npy")
# t2 = np.load("dataISEG_np/T2_fullAug.npy")
# mask = np.load("dataISEG_np/mask_fullAug.npy")
# t1, t2, mask = get_augmentation(t1, t2, mask)

create train_patch32_stride8.zip and test_patch32_stride8.zip

In [ ]:
count_train = 1
count_test = 1
TEST_PATH = "test_patch32_stride8/"
TRAIN_PATH = "train_patch32_stride8/"
for i in range(len(mask)):
    print("\r read in: ", i+1,"th volume", end="")
    label_patch = extract_patches(mask[i],PATCH_SIZE,EXTRACTION_STEP)
    valid_index = np.where(np.sum(label_patch, axis=(1, 2, 3, 4)) != 0)
    y = label_patch[valid_index]
    del label_patch

    t1_patch = extract_patches(t1[i], PATCH_SIZE, EXTRACTION_STEP)
    x_t1 = t1_patch[valid_index]
    del t1_patch

    t2_patch = extract_patches(t2[i], PATCH_SIZE, EXTRACTION_STEP)
    x_t2 = t2_patch[valid_index]
    del t2_patch

    x_t1_train, x_t1_test, x_t2_train, x_t2_test, y_train, y_test = train_test_split(x_t1, x_t2, y, test_size=0.1)
    for j in range(y_train.shape[0]):
        print("\rsave ",count_train,"th volume for train", end="")             
        imgT1 = nib.Nifti1Image( x_t1_train[j], np.eye(4))
        nib.save(imgT1, TRAIN_PATH+ 'T1_train_'+str(count_train)+'.nii.gz')  
        imgT2 = nib.Nifti1Image( x_t2_train[j], np.eye(4))
        nib.save(imgT2, TRAIN_PATH+ 'T2_train_'+str(count_train)+'.nii.gz')  
        img_mask = nib.Nifti1Image(y_train[j], np.eye(4))
        nib.save(img_mask, TRAIN_PATH+'mask_train_'+str(count_train)+'.nii.gz')    
        count_train += 1 

    for j in range(y_test.shape[0]):
        print("\rsave ",count_test,"th volume for test", end="")             
        imgT1 = nib.Nifti1Image(x_t1_test[j], np.eye(4))
        nib.save(imgT1, TEST_PATH+ 'T1_test_'+str(count_test)+'.nii.gz')  
        imgT2 = nib.Nifti1Image(x_t2_test[j], np.eye(4))
        nib.save(imgT2, TEST_PATH+ 'T2_test_'+str(count_test)+'.nii.gz')  
        img_mask = nib.Nifti1Image(y_test[j], np.eye(4))
        nib.save(img_mask, TEST_PATH+'mask_test_'+str(count_test)+'.nii.gz')    
        count_test += 1 
    del x_t1_train, x_t1_test, x_t2_train, x_t2_test, y_train, y_test, x_t1, x_t2, y


##model

In [ ]:
# class Swish(tf.keras.layers.Layer):
#     def __init__(self, name=None, **kwargs):
#         super().__init__(name=name, **kwargs)

#     def call(self, inputs, **kwargs):
#         return tf.nn.swish(inputs)
#     def get_config(self):
#         config = super().get_config()
#         config['name'] = self.name
#         return config
# def squeeze_excite_block(inputs, reduce_ratio=0.25,name_block=None):
#     filters = inputs.shape[-1]
#     num_reduced_filters= max(1, int(filters * reduce_ratio))
#     se = Lambda(lambda a: K.mean(a, axis=[1,2,3], keepdims=True))(inputs)

#     se = Conv3D(
#         num_reduced_filters,
#         kernel_size=(1, 1, 1), 
#         kernel_initializer='he_normal',
#         padding='same',
#         use_bias=True
#         )(se)
#     se = Swish()(se)
#     se = Conv3D(
#         filters,
#         kernel_size=(1, 1, 1),
#         kernel_initializer='he_normal',
#         padding='same',
#         use_bias=True
#         )(se)
#     se = Activation('sigmoid')(se)
#     if name_block is not None:
#         out = Multiply(name=name_block)([se, inputs])
#     else : 
#         out = Multiply()([se, inputs])
#     return out
# def conv_block(inputs, filters,kernel_size = (3,3,3), block_name=None):
#     x = inputs

#     x = Conv3D(filters, kernel_size, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
#     x = Swish()(x)

#     x = Conv3D(filters, kernel_size, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
#     x = Swish()(x)

#     x = squeeze_excite_block(x, name_block=block_name)

#     return x


# def convBn(inputs, filters,kernel_size = (1,1,1), block_name=None):
#     x = Conv3D(filters, kernel_size, padding="same", use_bias=False,kernel_initializer='he_normal')(inputs)
#     x = BatchNormalization()(x)
#     x = Swish()(x)
#     x = squeeze_excite_block(x, name_block=block_name)
#     return x

# def dense_block(inputs, filters, repetition):
#     x = inputs
#     for _ in range(repetition):
#         y = convBn(x, 4 * filters)
#         y = convBn(y, filters, (3,3,3))
#         x = Concatenate()([y,x])
#     return x

# def transition_block(inputs):
#     indim = inputs.shape[-1] //2 
#     x = Conv3D(indim , (1,1,1), padding="same", use_bias=False,kernel_initializer='he_normal')(inputs)
#     x = BatchNormalization()(x)
#     x = Swish()(x)
#     x = squeeze_excite_block(x)
#     x = Conv3D(x.shape[-1], (2,2,2),strides=(2,2,2), padding="same", use_bias=False,kernel_initializer='he_normal')(x)
#     x = Dropout(0.2)(x)
#     return x


# def ResidualBlock(inputs):
#     x = inputs
#     indim= inputs.shape[-1]
#     residual = Conv3D(indim,kernel_size=(1,1,1),padding='same')(x)
#     residual = BatchNormalization()(residual)
#     residual = Swish()(residual)
#     residual = Conv3D(indim,kernel_size=(3,3,3),padding='same')(residual)
#     residual = BatchNormalization()(residual)
#     residual = Swish()(residual)
#     residual = Conv3D(indim,kernel_size=(1,1,1),padding='same')(residual)
#     residual = BatchNormalization()(residual)
#     residual = Swish()(residual)

#     x        = BatchNormalization()(x)
#     #residual = Dropout(0.2)(residual)
#     out = x+ residual
    
#     return out
# def attention_module(inputs, skip, n_filter):
#     x = Conv3D(n_filter,(1, 1,1), padding='same', kernel_initializer='he_normal',use_bias=False)(inputs)
#     x = BatchNormalization()(x)
#     x = Conv3DTranspose(n_filter, (2,2,2), strides=(2, 2,2), padding='same',kernel_initializer = 'he_normal')(x)

#     x1= Conv3D(n_filter,(1, 1,1), padding='same', kernel_initializer='he_normal',use_bias=False)(skip)
#     x1 = BatchNormalization()(x1)

#     out = Swish()(x1+x)
#     out = Conv3D(1,(1, 1,1), padding='same', kernel_initializer='he_normal',use_bias=False)(out)
#     out = BatchNormalization()(out)
#     out = Activation('sigmoid')(out)

#     return out*skip

# def decoder_block(inputs, n_filter,skip=None):
#     x= Conv3DTranspose(n_filter, (2,2,2), strides=(2, 2, 2), padding='same',kernel_initializer = 'he_normal')(inputs)
#     out = x
#     if skip is not None :
#         attention = attention_module(inputs,skip, n_filter)
#         out = Concatenate()([x,attention])
#     # out = conv_block(out, n_filter)
#     out = convBn(out, out.shape[-1])

#     return out
# def seg_net(input_shape= (32,32,32,1), out_channels = 4):
#     # inp = Input(shape=input_shape)
#     # o = transition_block(16)(inp)
#     inpT1 = Input(shape=input_shape, name="inputT1")
#     inpT2 = Input(shape=input_shape, name="inputT2")

#     oT1 = convBn(inpT1, 16)
#     oT2 = convBn(inpT2, 16)
#     o = Concatenate()([oT1, oT2])
#     o = convBn(o, 32)

#     skip1 = dense_block(o, 16, repetition = 4)
#     o = transition_block(skip1)

#     skip2 = dense_block(o, 16, repetition = 4)
#     o = transition_block(skip2)

#     skip3 = dense_block(o, 16, repetition = 4)
#     o = transition_block(skip3)

#     skip4 = dense_block(o, 16, repetition = 4)
#     o = transition_block(skip4)


#     o = conv_block(o, 128)
#     o = Dropout(0.2)(o)
#     skip_connect = [skip4, skip3, skip2, skip1]

#     num_filters = [64, 64, 32, 16]
#     for i, f in enumerate(num_filters):
#         o = decoder_block(o, f,skip=skip_connect[i])
#         o = Dropout(0.1)(o)
    
#     o = Conv3D(out_channels,(1, 1 ,1), padding='same', kernel_initializer='he_normal')(o)
#     # o = BatchNormalization()(o)
#     if out_channels > 1 : 
#         output1 = Activation('softmax', name = "active")(o)
#         output2 = Activation('softmax', name = "levelset")(o)
#     else :
#         output1 = Activation('sigmoid', name = "active")(o)
#         output2 = Activation('sigmoid', name = "levelset")(o)
#     return Model([inpT1, inpT2], [output1, output2])

# # S=seg_net()
# # S.summary()

##train

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3,4])
    union = K.sum(y_true, axis=[1,2,3,4]) + K.sum(y_pred, axis=[1,2,3,4])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_CSF(y_true, y_pred, smooth = 1.0):
    output_standard = K.expand_dims(K.argmax(y_pred,axis=-1), axis=-1)
    output_CSF = tf.where(output_standard == 1, 1.0, 0.0)
    label_CSF = tf.where(y_true == 1, 1.0, 0.0)

    intersection = K.sum(label_CSF * output_CSF, axis=[1,2,3,4])
    union = K.sum(label_CSF, axis=[1,2,3,4]) + K.sum(output_CSF, axis=[1,2,3,4])
    return K.mean((2. * intersection + smooth) / (union + smooth), axis=0)

def dice_GM(y_true, y_pred, smooth = 1.0):
    output_standard = K.expand_dims(K.argmax(y_pred,axis=-1), axis=-1)
    output_GM = tf.where(output_standard == 2, 1.0, 0.0)
    label_GM = tf.where(y_true == 2, 1.0, 0.0)

    intersection = K.sum(label_GM * output_GM, axis=[1,2,3,4])
    union = K.sum(label_GM, axis=[1,2,3,4]) + K.sum(output_GM, axis=[1,2,3,4])
    return K.mean((2. * intersection + smooth) / (union + smooth), axis=0)

def dice_WM(y_true, y_pred, smooth = 1.0):
    output_standard = K.expand_dims(K.argmax(y_pred,axis=-1), axis=-1)
    output_WM = tf.where(output_standard == 3, 1.0, 0.0)
    label_WM = tf.where(y_true == 3, 1.0, 0.0)

    intersection = K.sum(label_WM * output_WM, axis=[1,2,3,4])
    union = K.sum(label_WM, axis=[1,2,3,4]) + K.sum(output_WM, axis=[1,2,3,4])
    return K.mean((2. * intersection + smooth) / (union + smooth), axis=0)

In [ ]:
def gradientLoss(y_pred,penalty = "l1"):

    dH = tf.math.abs(y_pred[:,1:,...] - y_pred[:,:-1,...])
    dW = tf.math.abs(y_pred[:,:,1:,...] - y_pred[:,:,:-1,...])
    dD = tf.math.abs(y_pred[...,1:,:] - y_pred[...,:-1,:])
    if penalty == "l2":
        dH = dH * dH
        dW = dW * dW
        dD = dD * dD

    loss =  tf.reduce_sum(dH) +  tf.reduce_sum(dW)+ tf.reduce_sum(dD)
    return loss
# def levelsetLoss(volume, y_pred, kernel_size = 3, beta = 1e-3):
#     kernel = tf.ones([kernel_size, kernel_size, kernel_size, 1, y_pred.shape[-1]],tf.float32) / (kernel_size**3)
      
#     pcentroid_1 = tf.nn.conv3d( volume * y_pred, kernel, strides=[1,1,1,1,1], padding='SAME')
#     pcentroid_2 =  tf.nn.conv3d(y_pred + 1e-7, kernel, strides=[1,1,1,1,1], padding='SAME')
#     pcentroid = pcentroid_1 / pcentroid_2
#     plevel = volume - pcentroid
#     loss = plevel * plevel * y_pred

#     lossLength = gradientLoss(y_pred)
#     # lossLength = 0
#     return tf.reduce_sum(loss) + beta * lossLength
def levelsetLossCVES(volume, y_pred,  beta = 1e-3):
    lossRegion = 0.0
    pcentroid = tf.reduce_sum(volume**2 * y_pred, (1,2,3),keepdims=True)/tf.reduce_sum(volume* y_pred+1e-10, (1,2,3),keepdims = True)   
    plevel = (volume - pcentroid) / pcentroid

    loss = plevel * plevel * y_pred
    lossRegion += tf.reduce_sum(loss)
    
    lossLength = gradientLoss(y_pred)
    return tf.reduce_sum(lossRegion) + beta * lossLength

def ActiveContourLoss(y_true, y_pred, smooth=1e-7):   
    maskOnehot = tf.one_hot(tf.squeeze(tf.cast(y_true,tf.uint8),axis=-1), depth = NUM_CLASS)
    # lossAC =  y_pred * (1-maskOnehot) + (1-y_pred)*maskOnehot
    # lossAC = 0
    #######################  this is Luac:##############
    lossAC =  - tf.cast(tf.math.log(1-y_pred+smooth),tf.float32) * (1-maskOnehot) - tf.cast(tf.math.log(y_pred+smooth),tf.float32)*maskOnehot

    return tf.reduce_mean(lossAC)

In [ ]:
# S = seg_net(input_shape=PATCH_SIZE)
# S.compile(optimizer=Nadam(learning_rate=5e-4), loss={"active":ActiveContourLoss, "levelset":levelsetLoss},\
#         loss_weights={"active": 1.0, "levelset": 1e-6},\
#         metrics={"active": [dice_CSF, dice_GM, dice_WM]})
mcp = ModelCheckpoint("./weightTraining/ckpt{val_active_dice_WM:.4f}.h5",mode='max',monitor='val_active_dice_WM',verbose=1,save_best_only=True,save_weights_only=True)
earlystop= EarlyStopping(monitor='val_active_dice_WM',patience=150,mode='max',verbose=1)
rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_active_dice_WM', factor=0.5, mode='max', patience=3, min_lr=1e-7, verbose=1)
csv_logger = CSVLogger("history.csv",append= True)


In [ ]:
train_dataset = patchSequence(batch_size= 16, augment = False)
test_dataset = patchSequence(batch_size= 32, typeData="test")

data length: 0
data length: 3384


In [ ]:
# S.save_weights("current.h5")
# S = seg_net(input_shape=PATCH_SIZE, keep_prob= 1)
# S.load_weights("current.h5")
# S.load_weights("weightTraining/ckpt0.9666.h5")

In [ ]:
# for keep_prob in np.linspace(start = 0.99, stop = 1, num=6):
#     print("dropout rate is:", keep_prob)
#     new_model = seg_net(input_shape=PATCH_SIZE, keep_prob = keep_prob)
#     new_model.set_weights(S.get_weights())
#     S = new_model
#     S.compile(optimizer=Nadam(learning_rate=2.5e-5), loss={"active":ActiveContourLoss, 
#                                                          "levelset":levelsetLossCVES},\
#             loss_weights={"active": 1.0, "levelset": 1e-9},\
#             metrics={"active": [dice_CSF, dice_GM, dice_WM]})
#     S.fit(train_dataset,epochs = 4, callbacks=[mcp, rlr,earlystop,csv_logger],\
#       validation_data=test_dataset)

In [ ]:
# S = seg_net(input_shape=PATCH_SIZE,keep_prob= 1)
# # S.load_weights("weightTraining2/ckpt0.9280.h5")
# for keep_prob in np.linspace(start = 1, stop = 0.9, num=51):
#     print("dropout rate is:", keep_prob)
#     new_model = seg_net(input_shape=PATCH_SIZE, keep_prob = keep_prob)
#     new_model.set_weights(S.get_weights())
#     S = new_model
#     S.compile(optimizer=Nadam(learning_rate=5e-4), loss={"active":ActiveContourLoss, 
#                                                          "levelset":levelsetLossCVES},\
#             loss_weights={"active": 1.0, "levelset": 1e-9},\
#             metrics={"active": [dice_CSF, dice_GM, dice_WM]})
#     S.fit(train_dataset,epochs = 1, callbacks=[mcp, rlr,earlystop,csv_logger],\
#       validation_data=test_dataset)

In [ ]:
S = seg_net(input_shape=PATCH_SIZE, keep_prob= 0.998)
S.compile(optimizer=Nadam(learning_rate=2.5e-5), loss={"active":ActiveContourLoss, 
                                                        "levelset":levelsetLossCVES},\
        loss_weights={"active": 1.0, "levelset": 1e-9},\
        metrics={"active": [dice_CSF, dice_GM, dice_WM]})

In [ ]:
S.load_weights("./weightTraining/ckpt0.9595.h5")

In [ ]:
S.evaluate(test_dataset)

52/52 [==============================] - 54s 724ms/step - loss: 0.0084 - active_loss: 0.0072 - levelset_loss: 1202888.3750 - active_dice_CSF: 0.9884 - active_dice_GM: 0.9718 - active_dice_WM: 0.9593


[0.008379052393138409,
 0.007176163140684366,
 1202888.375,
 0.98841392993927,
 0.9717748165130615,
 0.9593257904052734]

In [ ]:
S.fit(train_dataset,epochs = 1000, callbacks=[mcp, rlr,earlystop,csv_logger], \
      validation_data=test_dataset)

In [ ]:
# for layer in S.layers:
#     if isinstance(layer, DropBlock3D):
#         print(layer.block_size)
# S = load_model("model_U_Unet.h5",
#                custom_objects={"Swish":Swish,"DropBlock3D":DropBlock3D}, compile=False)
# S.compile(optimizer=Nadam(learning_rate=1.25e-4), loss={"active":ActiveContourLoss, 
#                                                         "levelset":levelsetLossCVES},\
#         loss_weights={"active": 1.0, "levelset": 1e-9},\
#         metrics={"active": [dice_CSF, dice_GM, dice_WM]})
# S.evaluate(test_dataset)

##psedo labeling


In [ ]:
def getValidIndex(label_constructed, patchT1, threshold):
    filter = np.max(label_constructed, axis=-1,keepdims=True)
    patch = extract_patches(filter, PATCH_SIZE, EXTRACTION_STEP)
    valid_index = []
    for index in range(patch.shape[0]):
        if (not False in np.unique(patch[index] > threshold)) and np.sum(patchT1[index]) != 0:
            valid_index.append(index)
    return valid_index
# IMG_SUBMIT_PATH = "/dataISEG/iSeg-2019-Validation/"
# LABEL_SUBMIT_PATH = "/dataISEG/iSeg-2017-Test-Prob/"
# for i in range(11,24):
#     print("\rsegment : {}th subject".format(i), end="")
#     imgT1 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T1.hdr").get_fdata()
#     imgT1 = normalize(imgT1)
#     imgT2 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T2.hdr").get_fdata()
#     imgT2 = normalize(imgT2)
#     patchT1 = extract_patches(imgT1, PATCH_SIZE, EXTRACTION_STEP)
#     patchT2 = extract_patches(imgT2, PATCH_SIZE, EXTRACTION_STEP)

#     pred = np.zeros((*patchT1.shape[:-1], NUM_CLASS))
#     pred[...,0] = 1
#     valid_index = np.where(np.sum(patchT1, axis=(1, 2, 3, 4)) != 0)
#     patchT1_valid = tf.convert_to_tensor(patchT1[valid_index], tf.float32)
#     patchT2_valid = tf.convert_to_tensor(patchT2[valid_index], tf.float32)
#     del patchT1, patchT2
#     temp = S.predict({"inputT1": patchT1_valid, 
#                         "inputT2": patchT2_valid}, batch_size = 32)[0]
#     gc.collect()
#     K.clear_session()
#     pred[valid_index] = temp
#     del valid_index, temp, patchT1_valid, patchT2_valid
#     label_constructed = reconstruct_volume_avg(pred, imgT1.shape[:-1], EXTRACTION_STEP)
#     del imgT1, imgT2, pred
#     label_save = nib.Nifti1Image(label_constructed, np.eye(4))
#     nib.save(label_save, LABEL_SUBMIT_PATH+"subject-"+str(i)+"-label.nii.gz")    
#     del label_constructed, label_save


In [ ]:
# count_train = 1
# PSEUDO_PATH = "/content/pseudo_data/"
# GET_LABEL_PATH  = "dataISEG/iSeg-2017-Test-Prob/"
# GET_IMAGE_PATH  = "dataISEG/iSeg-2017-Testing/"
# for i in range(11, 24):
#     label_constructed = nib.load(GET_LABEL_PATH+"subject-"+str(i)+"-label.nii.gz").get_fdata()
#     imgT1 = nib.load(GET_IMAGE_PATH+"subject-"+str(i)+"-T1.hdr").get_fdata()
#     imgT1 = normalize(imgT1)
#     patchT1 = extract_patches(imgT1, PATCH_SIZE, EXTRACTION_STEP)
#     valid_index = getValidIndex(label_constructed, patchT1, 0.5)

#     x_t1 = patchT1[valid_index]
#     del patchT1, imgT1
#     output_standard = np.expand_dims(np.argmax(label_constructed, axis=-1), axis=-1)
#     label_patch = extract_patches(output_standard,PATCH_SIZE,EXTRACTION_STEP)
#     label = label_patch[valid_index]
#     del label_patch, output_standard, label_constructed
#     imgT2 = nib.load(GET_IMAGE_PATH+"subject-"+str(i)+"-T2.hdr").get_fdata()
#     imgT2 = normalize(imgT2)
#     patchT2 = extract_patches(imgT2, PATCH_SIZE, EXTRACTION_STEP)
#     x_t2 = patchT2[valid_index]
#     del patchT2, imgT2 

#     for j in range(len(valid_index)):
#         print("\rsave ",count_train,"th volume for train", end="")             
#         imgT1 = nib.Nifti1Image( x_t1[j], np.eye(4))
#         nib.save(imgT1, PSEUDO_PATH+ 'T1_train_'+str(count_train)+'.nii.gz')  
#         imgT2 = nib.Nifti1Image( x_t2[j], np.eye(4))
#         nib.save(imgT2, PSEUDO_PATH+ 'T2_train_'+str(count_train)+'.nii.gz')  
#         img_mask = nib.Nifti1Image(label[j], np.eye(4))
#         nib.save(img_mask, PSEUDO_PATH+'mask_train_'+str(count_train)+'.nii.gz')    
#         count_train += 1 
# import shutil
# shutil.make_archive("pseudo_data", "zip","/content/pseudo_data") 

save  37820 th volume for train

##submit

In [ ]:
def convertLable2Submit(label):
    class_mapper_inv = {1 : 10, 2 : 150, 3 : 250}
    for key, value in class_mapper_inv.items():
        label = np.where(label==key, value, label)
    return label.astype(np.uint8)

In [ ]:
S = load_model("model_U_Unet.h5",
               custom_objects={"Swish":Swish,"DropBlock3D":DropBlock3D}, compile=False)

access performance on training-set

In [ ]:
# dice_csf = []
# dice_gm = []
# dice_wm = []
# for j in range(1,3):
#     imgT1 = nib.load("dataISEG/iSeg-2019-Training/subject-"+str(j)+"-T1.hdr").get_fdata()
#     imgT1 = normalize(imgT1)
#     imgT2 = nib.load("dataISEG/iSeg-2019-Training/subject-"+str(j)+"-T2.hdr").get_fdata()
#     imgT2 = normalize(imgT2)
#     label = nib.load("dataISEG/iSeg-2019-Training/subject-"+str(j)+"-label.hdr").get_fdata()
#     patchT1 = extract_patches(imgT1, PATCH_SIZE, EXTRACTION_STEP)
#     patchT2 = extract_patches(imgT2, PATCH_SIZE, EXTRACTION_STEP)

#     pred = np.zeros((*patchT1.shape[:-1], NUM_CLASS))
#     pred[...,0] = 1
#     valid_index = np.where(np.sum(patchT1, axis=(1, 2, 3, 4)) != 0)
#     patchT1_valid = tf.convert_to_tensor(patchT1[valid_index], tf.float32)
#     patchT2_valid = tf.convert_to_tensor(patchT2[valid_index], tf.float32)
#     del patchT1, patchT2
#     temp = S.predict({"inputT1": patchT1_valid, 
#                         "inputT2": patchT2_valid}, batch_size = 32)[0]
#     gc.collect()
#     K.clear_session()
#     pred[valid_index] = temp
#     del valid_index, temp, patchT1_valid, patchT2_valid
#     label_constructed = reconstruct_volume_avg(pred, imgT1.shape[:-1], EXTRACTION_STEP)
#     output_standard = tf.expand_dims(tf.argmax(label_constructed,axis=-1), axis=-1)
#     del imgT1, imgT2, pred, label_constructed
#     for i in range(1,4):
#         output_CSF = np.where(output_standard == i, 1.0, 0.0)
#         label_CSF = np.where(label == i, 1.0, 0.0)

#         intersection = K.sum(label_CSF * output_CSF, axis=[0,1,2,3])
#         union = K.sum(label_CSF, axis=[0,1,2,3]) + K.sum(output_CSF, axis=[0,1,2,3])
#         dice = ((2. * intersection + 1) / (union + 1))
#         if i==1:
#             dice_csf.append(dice)
#         elif i==2:
#             dice_gm.append(dice)
#         else : 
#             dice_wm.append(dice)
#         print(f"subject {j}: ", dice)
#     del intersection, union, dice, label, output_standard     
# del patchT1_valid, patchT2_valid
# print(np.mean(dice_csf), np.mean(dice_gm),np.mean(dice_wm))                    

create files to submit

In [ ]:
IMG_SUBMIT_PATH = "dataISEG/iSeg-2017-Testing/"
LABEL_SUBMIT_PATH = "dataISEG/iSeg-2017-Test-Label/"
for i in range(11,24):
    print("\rsegment : {}th subject".format(i), end="")
    imgT1 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T1.hdr").get_fdata()
    imgT1 = normalize(imgT1)
    imgT2 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T2.hdr").get_fdata()
    imgT2 = normalize(imgT2)
    patchT1 = extract_patches(imgT1, PATCH_SIZE, EXTRACTION_STEP)
    patchT2 = extract_patches(imgT2, PATCH_SIZE, EXTRACTION_STEP)

    pred = np.zeros((*patchT1.shape[:-1], NUM_CLASS))
    pred[...,0] = 1
    valid_index = np.where(np.sum(patchT1, axis=(1, 2, 3, 4)) != 0)
    patchT1_valid = tf.convert_to_tensor(patchT1[valid_index], tf.float32)
    patchT2_valid = tf.convert_to_tensor(patchT2[valid_index], tf.float32)
    del patchT1, patchT2
    temp = S.predict({"inputT1": patchT1_valid, 
                        "inputT2": patchT2_valid}, batch_size = 32)[0]
    gc.collect()
    K.clear_session()
    pred[valid_index] = temp
    del valid_index, temp, patchT1_valid, patchT2_valid
    label_constructed = reconstruct_volume_avg(pred, imgT1.shape[:-1], EXTRACTION_STEP)
    output_standard = np.expand_dims(np.argmax(label_constructed, axis=-1), axis=-1)
    output_standard = convertLable2Submit(output_standard)
    del imgT1, imgT2, pred, label_constructed

    label_save = nib.Nifti1Image(output_standard, np.eye(4))
    nib.nifti1.save(label_save, LABEL_SUBMIT_PATH+"subject-"+str(i)+"-label.hdr")
    del output_standard

segment : 23th subject